In [1]:
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2
from ortools.sat.python import cp_model
import numpy as np
from tqdm import tqdm
from sklearn.metrics import mean_absolute_percentage_error

In [2]:
X = np.load('X_20x20.npy')
Y = np.load('Y_20x20.npy')
a = (X[0]* 10000.00).astype(int)

In [3]:
def get_solution(manager, routing, solution):
    index = routing.Start(0)
    route = []
    while not routing.IsEnd(index):
        route.append(manager.IndexToNode(index))
        index = solution.Value(routing.NextVar(index))
    return solution.ObjectiveValue(), route

In [4]:
DISTANCE_MATRIX = a
num_nodes = len(DISTANCE_MATRIX)
all_nodes = range(num_nodes)

# Model.
model = cp_model.CpModel()

obj_vars = []
obj_coeffs = []

# Create the circuit constraint.
arcs = []
arc_literals = {}
for i in all_nodes:
    for j in all_nodes:
        if i == j:
            continue

        lit = model.new_bool_var("%i follows %i" % (j, i))
        arcs.append((i, j, lit))
        arc_literals[i, j] = lit

        obj_vars.append(lit)
        obj_coeffs.append(DISTANCE_MATRIX[i][j])

model.add_circuit(arcs)

# Minimize weighted sum of arcs. Because this s
model.minimize(sum(obj_vars[i] * obj_coeffs[i] for i in range(len(obj_vars))))

# Solve and print out the solution.
solver = cp_model.CpSolver()
#solver.parameters.log_search_progress = True
# To benefit from the linearization of the circuit constraint.
solver.parameters.linearization_level = 2

solver.solve(model)
# print(solver.response_stats())

current_node = 0
str_route = "%i" % current_node
route_is_finished = False
route_distance = 0
while not route_is_finished:
    for i in all_nodes:
        if i == current_node:
            continue
        if solver.boolean_value(arc_literals[current_node, i]):
            str_route += " %i" % i
            route_distance += DISTANCE_MATRIX[current_node][i]
            current_node = i
            if current_node == 0:
                route_is_finished = True
            break

print("Route:", str_route)
print("distance:", route_distance/10000)


Route: 0 5 17 19 2 12 3 10 6 4 13 16 18 7 11 15 1 14 8 9 0
distance: 244.5748


In [5]:
%timeit solver.solve(model)

23 ms ± 7.71 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
border = 60000
X_test = X[border:]
Y_test = Y[border:]

In [7]:
def solve_google(a):
    data = {}
    data["distance_matrix"] = a
    data["num_vehicles"] = 1
    data["depot"] = 0
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )
    routing = pywrapcp.RoutingModel(manager)
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    solution = routing.SolveWithParameters(search_parameters)
    return get_solution(manager, routing, solution)

In [8]:
ld = []
ll = []
N = X_test.shape[1]
for i in tqdm(range(X_test.shape[0])):
    a = (X_test[i] * 10000.00).astype(int)
    route = Y_test[i]
    distance = sum(a[route[j],route[j+1]] for j in range(N-1))+a[route[-1],route[0]]
    ldist, _ = solve_google(a)
    ld.append(distance)
    ll.append(ldist)

100%|██████████| 3000/3000 [00:25<00:00, 118.29it/s]


In [9]:
Y_true = np.array(ld)
Y_l = np.array(ll)
mean_absolute_percentage_error(Y_true, Y_l)

0.031011210607327238

In [10]:
%timeit solve_google(a)

7.7 ms ± 11.5 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
